In [2]:
import pandas as pd
import portable_spreadsheet as ps
import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter
import numpy as np
from pathlib import Path
import itertools
from pandasql import *
pysqldf = lambda q:sqldf(q,globals())
import tkinter as tk
from tkinter import filedialog
import os, re
from re import match

In [3]:
spec_path = "C:/Users/priyeshdixitkumar.su/Desktop/autospec/"
os.chdir(spec_path)
os.getcwd()

'C:\\Users\\priyeshdixitkumar.su\\Desktop\\autospec'

In [4]:
spec_gen = re.sub(r"\s+", "", input('Input specification to be created: ').strip().upper(), flags=re.UNICODE)
spec_gen

Input specification to be created: FAHYG


'FAHYG'

In [5]:
srdm_list = [filename for filename in os.listdir('.') if filename.startswith(spec_gen) and filename.endswith('.xlsx')]
srdm_list

df_srdm=[]    #Create an empty list to append the data for all domains in fval_list
cols = [17, 14, 19, 13]

for i in range(len(srdm_list)):
    srdm00 = srdm_list[i]
    srdm01 = pd.read_excel(srdm00, sheet_name=3, skiprows=1,
                      usecols=cols, names=['SType','SLength','SName','SLabel'])
    srdm01 = srdm01.loc[srdm01['SName'].str.contains('_')]
    srdm01['SType']= np.where(srdm01['SType'].str.upper() == 'VARCHAR2','Character',srdm01['SType'].str.title())
    n_col = (srdm01['SName'].str.split("_").apply(len)).max()
    collis = []
    for i in range(n_col): 
        collis.append('V'+str(i))
    srdm01[collis] = srdm01['SName'].str.split('_', expand=True)
    df_srdm.append(srdm01)
df_srdm = pd.concat(df_srdm)
df_srdm

domainlist = np.unique([x[:2] for x in df_srdm['SName'] if isinstance(x, str) and x.count('_') == 1]).tolist()
domainlist
print(srdm_list)
print(df_srdm)

['FAHYG018_SRDM_SPECIFICATION_V2.1.xlsx']
         SType  SLength                   SName  \
11   Character     26.0        FACAT_HYGCAT_018   
12   Character     40.0               FAOBJ_018   
13   Character     40.0              FAEVAL_018   
14   Character     20.0            Y_FASPID_018   
15   Character     64.0         Y_FASPID_018_ND   
..         ...      ...                     ...   
105  Character     40.0     FAORRES_DMERRWT_018   
106  Character     10.0   FAORRES_EXERCSE_018_C   
107  Character     40.0     FAORRES_EXERCSE_018   
108  Character     10.0  FAORRES_OTPTEVNT_018_C   
109  Character     40.0    FAORRES_OTPTEVNT_018   

                                               SLabel       V0        V1  \
11   Category for Findings About # Hypoglycemia Event    FACAT    HYGCAT   
12                          Object of the Observation    FAOBJ       018   
13                                          Evaluator   FAEVAL       018   
14                         Sponsor-Define

In [6]:
fval = input('Input Domain Names Separated By Commas: ')
fval_list = np.unique([f.strip().upper() for f in fval.split(',') if fval]).tolist()
fval_list
domains = np.unique(domainlist+fval_list).tolist()
domains

Input Domain Names Separated By Commas: 


['CM', 'FA', 'LB', 'ML']

In [8]:
#Reading the NextGen Meta file to get the list of variables (Names) to be added to specification.
nextgen = pd.read_excel('NextGEN SDTM Metadata.xlsx', sheet_name="Elements", usecols=['Order','Dataset','Name','Core'])
nextgen = nextgen[nextgen['Dataset'].isin(domains)]
nextgen.reset_index(drop=True)
nextgen.head()
#len(nextgen.index) == 0

,Order,Dataset,Name,Core
151,1,CM,STUDYID,Required
152,2,CM,DOMAIN,Required
153,3,CM,USUBJID,Expected
154,4,CM,POOLID,Model Permissible
155,5,CM,SPDEVID,Model Permissible


In [16]:
sdtm32 = pd.read_excel('SDTMIG.xlsx', sheet_name="SDTMIG v3.2",header=None, skiprows=1,
                        names=['Version','Order','Class','Dataset','Vname','Name','Label','Type','CodeListRef','Role','Description','Core'])
sdtm33 = pd.read_excel('SDTMIG.xlsx', sheet_name="SDTMIG v3.3",header=None, skiprows=1,
                        names=['Version','Order','Class','Dataset','Vname','Name','Label','Type','CodeListRef','Role','Description','Core'])

In [8]:
sdtm32 = pd.read_excel('Consiladated Meta.xls', sheet_name="P21SDTMIG3.2_Variables"
                       ,usecols=['Order','Dataset','Name','Label', 'Type', 'Core', 'Codelist', 'Origin'])
sdtm32 = sdtm32[sdtm32.Core.isin(['Required', 'Expected', 'Permissible', 'Model Permissible'])]
sdtm33 = pd.read_excel('Consiladated Meta.xls', sheet_name="P21SDTMIG3.3_Variables"
                       ,usecols=['Order','Dataset','Name','Label', 'Type', 'Core', 'Codelist', 'Origin'])
sdtm33 = sdtm33[sdtm33.Core.isin(['Required', 'Expected', 'Permissible', 'Model Permissible'])]

labelmeta = pd.read_excel('Consiladated Meta.xls', sheet_name="MetaLabelsCompare")

In [9]:
#Get Unique list of domains in SDTM IG 3.2
domain_32 = []
domain_32 = sdtm32['Dataset'].unique()
domain_32 = [x for x in domain_32 if str(x) != 'nan']

#Get Unique list of domains in SDTM IG 3.3
domain_33=[]
domain_33 = sdtm33['Dataset'].unique()
domain_33 = [x for x in domain_33 if str(x) != 'nan']

#List Domains in SDTM IG 3.3 which are not available in SDTM IG 3.2
in33_not_in32 = []
in33_not_in32=[x for x in domain_33 if x not in set(domain_32)]

#Keep Only those domains that do not exist in 3.2 and available in SDTM IG 3.3 
sdtm33 = sdtm33[sdtm33.Dataset.isin(in33_not_in32)]

fval_in32 = [x for x in domains if x in set(domain_32)] #Domain in 3.2
fval_in33 = [x for x in domains if x in set(in33_not_in32)] #Domain in 3.3
neither_32_nor_33 = [x for x in domains if x not in set(domain_32 and domain_33)] #Domains in neither 3.2 nor 3.3

#Consolidated SDTM IG 3.2 and 3.3 (Only those domains that do not exist in 3.2)
sdtm_meta = sdtm32.append(sdtm33,ignore_index=True)

#neither_32_nor_33 =  not (fval_in32 or fval_in33)
print(fval_in32)
print(fval_in33)
print(neither_32_nor_33)
sdtm_meta

['CM', 'FA', 'LB', 'RELREC']
['ML']
[]


,Order,Dataset,Name,Label,Type,Core,Codelist,Origin
0,1,AE,STUDYID,Study Identifier,text,Required,NaN,Protocol
1,2,AE,DOMAIN,Domain Abbreviation,text,Required,DOMAIN,Assigned
2,3,AE,USUBJID,Unique Subject Identifier,text,Expected,NaN,Derived
3,4,AE,POOLID,Pool Identifier,text,Model Permissible,NaN,NaN
4,5,AE,SPDEVID,Sponsor Device Identifier,text,Model Permissible,NaN,NaN
...,...,...,...,...,...,...,...,...
4363,99,UR,MIDS,Disease Milestone Instance Name,text,Model Permissible,NaN,NaN
4364,100,UR,RELMIDS,Temporal Relation to Milestone Instance,text,Model Permissible,NaN,NaN
4365,101,UR,MIDSDTC,Disease Milestone Instance Date/Time,datetime,Model Permissible,NaN,NaN
4366,102,UR,URSTINT,Planned Start of Assessment Interval,datetime,Model Permissible,NaN,NaN


In [10]:
sdtm_00=[]    #Create an empty list to append the data for all domains in fval_list
for i in range(len(domains)):
    domain = domains[i]
    sdtm_01 = sdtm_meta[sdtm_meta['Dataset'] == domain]   #To filter on domain
    sdtm_01['Type']=np.where(sdtm_01['Type']=='text','Character','Number')
    sdtm_00.append(sdtm_01) 
sdtm_00 = pd.concat(sdtm_00)
sdtm_00
#with pd.ExcelWriter(path + 'spec.xlsx',engine='xlsxwriter', mode='a') as writer:
 #       sdtm32_02.to_excel(writer, sheet_name="sdtm5",index=False)

<ipython-input-10-2c84a11bc819>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdtm_01['Type']=np.where(sdtm_01['Type']=='text','Character','Number')


,Order,Dataset,Name,Label,Type,Core,Codelist,Origin
326,1,CM,STUDYID,Study Identifier,Character,Required,NaN,Protocol
327,2,CM,DOMAIN,Domain Abbreviation,Character,Required,DOMAIN,Assigned
328,3,CM,USUBJID,Unique Subject Identifier,Character,Expected,NaN,Derived
329,4,CM,POOLID,Pool Identifier,Character,Model Permissible,NaN,NaN
330,5,CM,SPDEVID,Sponsor Device Identifier,Character,Model Permissible,NaN,NaN
...,...,...,...,...,...,...,...,...
2693,4,RELREC,POOLID,Pool Identifier,Character,Model Permissible,NaN,NaN
2694,5,RELREC,IDVAR,Identifying Variable,Character,Required,NaN,Assigned
2695,6,RELREC,IDVARVAL,Identifying Variable Value,Character,Expected,NaN,Assigned
2696,7,RELREC,RELTYPE,Relationship Type,Character,Expected,RELTYPE,Assigned


In [29]:
#Read The SDTM 3.2 IG in excel format and get only the desired columns.
#sdtm32 = pd.read_excel('sdtm_3.2_excel.xls', sheet_name="SDTM",header=None, skiprows=1, 
#                        names=['Order','Class','Dataset','Vname','Name','Label','Type','CodeListRef','Role','Description','Core'])
sdtm_meta = sdtm_meta[['Class','Dataset','Name','Core','Label','Type', 'CodeListRef', 'Description']]

df_00 = ['All Classes']    #Create a list for filteration criteria based on 'Class' column.
sdtm32_02=[]    #Create an empty list to append the data for all domains in fval_list
for i in range(len(fval_list)):
    domain = fval_list[i]
    sdtm32_00 = sdtm32['Class'][sdtm32['Dataset'].isin([domain])].unique()    #To get the 'Class' of respective domain from fval.
    df_00.append([x+'-General' for x in sdtm32_00][0])    #To get the Class with corresponding '-General' for filteration
    # To filter the dataframe with required data and class in 'All Classes' and corresponding '-General' Class.
    sdtm32_01 = sdtm32[np.logical_and(sdtm32['Dataset'].isna(), sdtm32['Class'].isin(df_00))]
    sdtm32_01 =pd.concat([sdtm32_01,sdtm32[sdtm32['Dataset'] == domain]])   #To filter on domain
    sdtm32_01['Name']=sdtm32_01['Name'].str.replace('--',domain)    #replace'--' with Domain name in "Name" Column
    sdtm32_01['Dataset']=np.where(sdtm32_01['Dataset'].isnull(),domain,sdtm32_01['Dataset'])
    sdtm32_01['Core']=np.where(sdtm32_01['Core'].isnull(),'Perm',sdtm32_01['Core'])
    sdtm32_01['Type']=np.where(sdtm32_01['Type']=='Num','Number','Character')
    sdtm32_02.append(sdtm32_01) 
    df_00=['All Classes']
sdtm32_02 = pd.concat(sdtm32_02)
#keep the last duplicate value per Dataset & Name
sdtm32_02.drop_duplicates(subset=['Dataset','Name'], keep='last', inplace=True)
#To Remove '()' from 'CodeListRef' Column.
sdtm32_02['CodeListRef'] = sdtm32_02['CodeListRef'].str.replace(r'[^\w\s]+', '')
sdtm32_02['CodeListRef']= np.where(sdtm32_02['Name'].str.upper() == 'EPOCH','EPOCH',sdtm32_02['CodeListRef'])
sdtm32_02['CodeListRef']= np.where(sdtm32_02['CodeListRef'].str.upper().isin(['MEDDRA','ISO 8601','ISO 3166-1 Alpha-3']),np.nan,sdtm32_02['CodeListRef'])
sdtm32_02
#with pd.ExcelWriter(path + 'spec.xlsx',engine='openpyxl', mode='a') as writer:
 #       sdtm32_02.to_excel(writer, sheet_name="sdtm5",index=False)

KeyError: "['Class', 'Description', 'Type', 'CodeListRef'] not in index"

In [ ]:
#sdtm32_01['Dataset']=sdtm32_01['Dataset'].replace(np.nan,domain)    #replace'--' with Domain name  
#sdtm32_01['Dataset']=[domain if type(y) !=str else y for y in sdtm32_01['Dataset'].tolist()]
#sdtm32_001 = sdtm32_01.fillna({'Dataset': domain },inplace=True)
#sdtm32_001
#str.translate({ord(i): None for i in '()'})
#sdtm32_02['Core'].map(lambda x: re.sub(r'\W+', '', x))
#sdtm32_02['CodeListRef'].str.replace(r'[^\w\s]+'"", '').unique()
#df3 = df.merge(df1,on=["Name"],how='left',suffixes=('_y',None)) 
#df['Description'] = df1[df1['Name'].isin(df['Name'].to_numpy())]['Description']
#srdm03[collis] = srdm01['Name'].apply(lambda x: pd.Series(str(x).split("_",)))
#srdm02 = srdm01.applymap(lambda x: x.strip() if isinstance(x, str) else x)
#df0001 = sdtm32_02.applymap(lambda x: x.strip() if isinstance(x, str) else x)
#df_sdtm = pd.merge(df0001,srdm02,left_on = 'Name',right_on='SName',how = 'left')
#with pd.ExcelWriter(path + 'spec.xlsx',engine='openpyxl', mode='a') as writer:
#       df_sdtm.to_excel(writer, sheet_name="sdtm5",index=False)
#df0001 = sdtm32_02.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    #sdtm32_001 = sdtm32_01.fillna({'Dataset': domain })    #replace Missing Dataset values with corresponding Domain name
    #sdtm32_01['Dataset']=sdtm32_01['Dataset'].replace(np.nan,domain)
    #sdtm32_01['Dataset']=sdtm32_01['Dataset'].mask(pd.isnull, domain)
#[f(x) if condition else g(x) for x in sequence]
#[f(x) for x in sequence if condition]
#[unicode(x.strip()) if x is not None else '' for x in row]
#domain_32.apply(lambda x: set(fval_list).issubset(x))
#for f in os.listdir('.')
#    if re.match(spec_gen, f):
#        print(f)
#domainlist1 = ', '.join(df_srdm['V1']).split(', ')
#domainlist2 = [f.strip() for f in domainlist1 if re.match('^\d*$', domainlist1)].tolist()
#filtered_values = list(filter(lambda v: match('^\d*$', v), ', '.join(df_srdm['V1']).split(', ')))
#filtered_values
#domainlist = [x[:2] if isinstance(x, str) and x.count('_') == 1 else np.nan for x in df_srdm['SName']]
#domain_32.apply(lambda x: set(fval_list).issubset(x))
#in33=[x for x in domain_32 if x not in set(domain_33)]
#in32=sum(x == y for x, y in zip(fval_list, domain_32)) == len(fval_list) == len(domain_32)
#in33 = [e for e in domain_32 if e in domain_33 and (domain_33.pop(domain_33.index(e)) or True)]
#print(fval_in32)

In [22]:
df_vcom = pd.read_excel('VOL3SRDM_SDTM_MAPPING_SPECIFICATION_COMM.xlsx', sheet_name="COMM")
for i in range(len(domains)):
    domain = domains[i]
    df_vcom['Domain'] = domain
    df_vcom['Name']=df_vcom['Name'].str.replace('__',domain)
df_vcom

,Name,Description,CodeListRef,Label,Length,Sequence,Supplimentary,Comments,Type,Origin,Core,ProgrammerRule,Submission,SRDMOrigin,Alias,Domain
0,STUDYID,Unique identifier for a study.,NaN,Study Identifier,200.0,1.0,0.0,0.0,Character,Protocol,Req,Rename STUDYNAME as STUDYID,NaN,STUDYNAME,NaN,ML
1,DOMAIN,Two-character abbreviation for the domain.,DOMAIN,Domain Abbreviation,2.0,2.0,0.0,0.0,Character,Assigned,Req,Direct move from seq_gen_key.domain where doma...,NaN,NaN,NaN,ML
2,USUBJID,Identifier used to uniquely identify a subject...,NaN,Unique Subject Identifier,60.0,3.0,0.0,0.0,Character,Derived,Req,"Derive from DM.USUBJID using generic utility ""...",Unique subject identifier derived using COMP.U...,NaN,NaN,ML
3,CMSEQ,Sequence Number given to ensure uniqueness of ...,NaN,Sequence Number,8.0,4.0,0.0,0.0,Number,Derived,Req,"Derive from generic utility ""util_srdm_seq_ke...",Sequence Number derived using COMP.SEQ,NaN,NaN,ML
4,VISITNUM,1. Clinical encounter number.\n\n2. Numeric ve...,NaN,Visit Number,8.0,29.0,0.0,0.0,Number,Assigned,Exp,Rename VISITID as VISITNUM,Assigned from source database. COMM.VISITNUM,VISITID,NaN,ML
5,VISIT,Protocol-defined description of clinical encou...,NaN,Visit Name,80.0,30.0,0.0,0.0,Character,Assigned,Perm,Rename VISITREFNAME as VISIT\nNote: If Unplann...,Assigned from source database. COMM.VISIT,VISITREFNAME,NaN,ML
6,EPOCH,Epoch associated with the Element in the plann...,EPOCH,Epoch,50.0,31.0,0.0,0.0,Character,Derived,Perm,"Derive using generic utility ""util_generate_ep...",Epoch derivation using COMP.EPOCH,NaN,NaN,ML
7,CMDY,"1. Study day of specimen collection, measured ...",NaN,Study Day of Specimen Collection,8.0,33.0,0.0,0.0,Number,Derived,Perm,"Derive from generic utility ""util_vol3srdm_cal...",DY derivation using COMP.DY,NaN,NaN,ML
8,CMEVLINT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Direct move from corresponding source variable...,NaN,NaN,NaN,ML
9,CMEVLINTX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Direct move from corresponding source variable...,NaN,NaN,NaN,ML


In [15]:
#QSPRM074_SRDM_Specification_V1.0
#Read The SRDM Specification in excel format and get only the desired columns.
cols = [17, 19, 14, 13]
srdm00 = pd.read_excel('FAHYG018_SRDM_SPECIFICATION_V2.1.xlsx', sheet_name="SM_FAHYG018", skiprows=1,
                      usecols=cols, names=['SType','SLength','SName','SLabel'])
srdm01 = srdm00.loc[srdm00['SName'].str.contains('_')]
srdm01['SType']= np.where(srdm01['SType'].str.upper() == 'VARCHAR2','Character',srdm01['SType'].str.title())
srdm01

<ipython-input-15-02149176d018>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  srdm01['SType']= np.where(srdm01['SType'].str.upper() == 'VARCHAR2','Character',srdm01['SType'].str.title())


,SType,SLength,SName,SLabel
11,Character,26.0,FACAT_HYGCAT_018,Category for Findings About # Hypoglycemia Event
12,Character,40.0,FAOBJ_018,Object of the Observation
13,Character,40.0,FAEVAL_018,Evaluator
14,Character,20.0,Y_FASPID_018,Sponsor-Defined Identifier
15,Character,64.0,Y_FASPID_018_ND,Sponsor-Defined Identifier # ND
...,...,...,...,...
105,Character,40.0,FAORRES_DMERRWT_018,Diabetes Medication Wrong Time
106,Character,10.0,FAORRES_EXERCSE_018_C,Exercise # C
107,Character,40.0,FAORRES_EXERCSE_018,Exercise
108,Character,10.0,FAORRES_OTPTEVNT_018_C,Other Precipitating Event # C


In [16]:
n_col = (srdm01['SName'].str.split("_").apply(len)).max()
n_col
collis = []
for i in range(n_col): 
    collis.append('V'+str(i))
#collis.append('V'+str(i)) for i in range(n_col)
#[f(x) for x in sequence if condition]
srdm01[collis] = srdm01['SName'].str.split('_', expand=True)
srdm01

C:\Users\priyeshdixitkumar.su\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,SType,SLength,SName,SLabel,V0,V1,V2,V3,V4
11,Character,26.0,FACAT_HYGCAT_018,Category for Findings About # Hypoglycemia Event,FACAT,HYGCAT,018,None,None
12,Character,40.0,FAOBJ_018,Object of the Observation,FAOBJ,018,None,None,None
13,Character,40.0,FAEVAL_018,Evaluator,FAEVAL,018,None,None,None
14,Character,20.0,Y_FASPID_018,Sponsor-Defined Identifier,Y,FASPID,018,None,None
15,Character,64.0,Y_FASPID_018_ND,Sponsor-Defined Identifier # ND,Y,FASPID,018,ND,None
...,...,...,...,...,...,...,...,...,...
105,Character,40.0,FAORRES_DMERRWT_018,Diabetes Medication Wrong Time,FAORRES,DMERRWT,018,None,None
106,Character,10.0,FAORRES_EXERCSE_018_C,Exercise # C,FAORRES,EXERCSE,018,C,None
107,Character,40.0,FAORRES_EXERCSE_018,Exercise,FAORRES,EXERCSE,018,None,None
108,Character,10.0,FAORRES_OTPTEVNT_018_C,Other Precipitating Event # C,FAORRES,OTPTEVNT,018,C,None


In [80]:
s_sdtm = pd.merge(sdtm32_02,srdm01,left_on = 'Name',right_on='V0',how = 'left') 
#df_002 = pysqldf("select a.*, b.* from sdtm32_02 as a left join srdm01 as b on a.Name = b.V0;")
#with pd.ExcelWriter(path + 'spec.xlsx',engine='openpyxl', mode='a') as writer:
       #df_002.to_excel(writer, sheet_name="sdtm8",index=False)
s_sdtm.head()

,Class,Dataset,Name,Core,Label,Type,CodeListRef,Description,SType,SLength,SName,SLabel,V0,V1,V2,V3,V4
0,All Classes,CM,POOLID,Perm,Pool Identifier,Character,NaN,An identifier used to identify a result from a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,All Classes,CM,SPDEVID,Perm,Sponsor Device Identifier,Character,NaN,Sponsor-defined identifier for a device,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,All Classes,CM,CMREFID,Perm,Reference ID,Character,NaN,Optional internal or external identifier such ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,All Classes,CM,CMLNKID,Perm,Link ID,Character,NaN,Identifier used to link related records across...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,All Classes,CM,CMLNKGRP,Perm,Link Group ID,Character,NaN,Identifier used to link related records across...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
def f(row):
    if row['Name'] == row['V0']:
        if np.logical_or(row['Type'] == row['SType'], row['Label'] == row['SLabel']):
            val = "Rename " + row['SName'] + " as " + row['Name']+"|"+row['SName']       
        else:
            val = "Direct Move from "+row['SName']+" to " + row['Name']+"|"+row['SName'] 
        return val
s_sdtm['PRuleSOrgn'] = s_sdtm.apply(f, axis=1)
#s_sdtm['PRule'] = s_sdtm['PRuleSOrgn'].str.split('|', expand=True)[0]
#s_sdtm['SOrgn'] = s_sdtm['PRuleSOrgn'].str.split('|', expand=True)[1]
s_sdtm['PRule'] = np.where(s_sdtm['PRuleSOrgn'].isnull()
                           ,s_sdtm['Name'].map(df_vcom.set_index('Name')['ProgrammerRule'])
                           ,s_sdtm['PRuleSOrgn'].str.split('|', expand=True)[0])
s_sdtm['SOrgn'] = np.where(s_sdtm['PRuleSOrgn'].isnull()
                           ,s_sdtm['Name'].map(df_vcom.set_index('Name')['SRDMOrigin'])
                           ,s_sdtm['PRuleSOrgn'].str.split('|', expand=True)[1])
s_sdtm['SubCol'] = s_sdtm['Name'].map(df_vcom.set_index('Name')['Submission'])
s_sdtm['Origin'] = s_sdtm['Name'].map(df_vcom.set_index('Name')['Origin'])
s_sdtm['Length'] = s_sdtm['Name'].map(df_vcom.set_index('Name')['Length'])

#elif df_002['Name'] == df_002['V1']:
#    if df_002['V0'].str.startswith('SUPP'):
#        df_002['Prule'] = "QVAL = Direct Move from"+df_002['SName'].str+" Variable " + "\n" + "QORIG='eDT'" + "\n" + "QEVAL=''"
s_sdtm[~s_sdtm['SOrgn'].isnull()]

,Class,Dataset,Name,Core,Label,Type,CodeListRef,Description,SType,SLength,...,V1,V2,V3,V4,PRuleSOrgn,PRule,SOrgn,SubCol,Origin,Length
5,All Classes,CM,VISITNUM,Perm,Visit Number,Number,NaN,Clinical encounter number. Numeric version of ...,NaN,NaN,...,NaN,NaN,NaN,NaN,None,Rename VISITID as VISITNUM,VISITID,Assigned from source database. COMM.VISITNUM,Assigned,8.0
6,All Classes,CM,VISIT,Perm,Visit Name,Character,NaN,Protocol-defined description of a clinical enc...,NaN,NaN,...,NaN,NaN,NaN,NaN,None,Rename VISITREFNAME as VISIT\nNote: If Unplann...,VISITREFNAME,Assigned from source database. COMM.VISIT,Assigned,80.0
35,Interventions,CM,STUDYID,Req,Study Identifier,Character,NaN,Unique identifier for a study.,NaN,NaN,...,NaN,NaN,NaN,NaN,None,Rename STUDYNAME as STUDYID,STUDYNAME,NaN,Protocol,200.0
44,Interventions,CM,CMCAT,Perm,Category for Medication,Character,NaN,Used to define a category of medications/treat...,Character,18.0,...,018,None,None,None,Rename CMCAT_018 as CMCAT|CMCAT_018,Rename CMCAT_018 as CMCAT,CMCAT_018,NaN,NaN,NaN
47,Interventions,CM,CMOCCUR,Perm,CM Occurrence,Character,NY,When the use of specific medications is solici...,Character,10.0,...,ORAL,018,C,None,Rename CMOCCUR_ORAL_018_C as CMOCCUR|CMOCCUR_O...,Rename CMOCCUR_ORAL_018_C as CMOCCUR,CMOCCUR_ORAL_018_C,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,Findings,LB,LBORRESU,Exp,Original Units,Character,UNIT,Original units in which the data were collecte...,Character,10.0,...,018,C,None,None,Rename LBORRESU_018_C as LBORRESU|LBORRESU_018_C,Rename LBORRESU_018_C as LBORRESU,LBORRESU_018_C,NaN,NaN,NaN
259,Findings,LB,LBORRESU,Exp,Original Units,Character,UNIT,Original units in which the data were collecte...,Character,40.0,...,018,None,None,None,Rename LBORRESU_018 as LBORRESU|LBORRESU_018,Rename LBORRESU_018 as LBORRESU,LBORRESU_018,NaN,NaN,NaN
273,Findings,LB,LBSPEC,Perm,Specimen Type,Character,SPECTYPE,Defines the type of specimen used for a measur...,Character,5.0,...,018,None,None,None,Rename LBSPEC_018 as LBSPEC|LBSPEC_018,Rename LBSPEC_018 as LBSPEC,LBSPEC_018,NaN,NaN,NaN
281,Findings,LB,VISITNUM,Exp,Visit Number,Number,NaN,1. Clinical encounter number. \n2. Numeric ver...,NaN,NaN,...,NaN,NaN,NaN,NaN,None,Rename VISITID as VISITNUM,VISITID,Assigned from source database. COMM.VISITNUM,Assigned,8.0


In [83]:
for i in range(len(fval_list)):
    sheet = fval_list[i]
    df = pd.read_excel('spec.xlsx', sheet_name="Domainname")
    df1 = s_sdtm[s_sdtm['Dataset'] == sheet]
    df['Name'] = nextgen[nextgen['Dataset'] == sheet]['Name'].to_numpy()
    df['NgCore'] = df['Name'].map(nextgen[nextgen['Dataset'] == sheet].set_index('Name')['Core'])
    df['Sequence'] = df['Name'].map(nextgen[nextgen['Dataset'] == sheet].set_index('Name')['Order'])
    df['Description'] = df['Name'].map(df1.set_index('Name')['Description'])
    df['Core'] = df['Name'].map(df1.set_index('Name')['Core'])
    df['CodeListRef'] = df['Name'].map(df1.set_index('Name')['CodeListRef'])
    df['Label'] = df['Name'].map(df1.set_index('Name')['Label'])
    df['Type'] = df['Name'].map(df1.set_index('Name')['Type'])
    df['Supplimentary']=df['Supplimentary'].mask(pd.isnull, 0)
    df['Comments']=df['Comments'].mask(pd.isnull, 0)
    df['ProgrammerRule'] = df['Name'].map(df1.set_index('Name')['PRule'])
    df['SRDMOrigin'] = df['Name'].map(df1.set_index('Name')['SOrgn'])
    df['Origin'] = df['Name'].map(df1.set_index('Name')['Origin'])
    df['Length'] = df['Name'].map(df1.set_index('Name')['Length'])
    #df.style.apply(highlight_row, charval='Model Permissible', column=['NgCore'], axis=1)
    with pd.ExcelWriter('spec.xlsx',engine='openpyxl', mode='a') as writer:
        df.to_excel(writer, sheet_name=sheet,index=False)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
for i in range(len(fval_list)):
    sheet = fval_list[i]        
    workbook = openpyxl.load_workbook(path + 'spec.xlsx')
    worksheet = workbook[sheet]
    for row in rows:
        cell_A = row[:1][0]
        cell_A.alignment = Alignment(wrapText=True)
        
            MIN_WIDTH = 20
    for j, column_cells in enumerate(worksheet.columns, start=1):
        width = (
            length
            if (length := max(len(str(cell_value) if (cell_value := cell.value) is not None else "")
                              for cell in column_cells)) >= MIN_WIDTH
            else MIN_WIDTH
            )
    worksheet.column_dimensions[get_column_letter(j)].width = width

In [ ]:
writer = pd.ExcelWriter('Trial Version.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)
workbook=writer.book
worksheet = writer.sheets['Sheet1']

format = workbook.add_format({'text_wrap': True})

# Setting the format but not setting the column width.
worksheet.set_column('A:B', None, format)

writer.save()

In [ ]:
def highlight_row(s, charval, column):
    is_charval = pd.Series(data=False, index=s.index)
    is_charval[column] = s.loc[column] == charval
    return ['background-color: yellow' if is_charval.any() else '' for v in is_charval]

1.) Creating a template same as 'Domainname' in case domain asked by User is not available in SDTM IG 3.2/3.3/OC Meta/VOL3 Meta.
#Create fval based on SRDM spec rather than
2.) Compare and keep the maximum length out of SRDM Spec/OC Meta/Vol3 Meta
3.) Populate all the columns of Common variables in all the specifications/Sheet asked by user.
4.) Defensive coding right form start
5.) Direct Map/Rename Variables automation
6.) Y_ variable automation

In [ ]:
#def write_sheet(val, df=None):
#    if df == None: 
#        df = pd.DataFrame(columns= ['Name','Description','CodeListRef',
#                                'Label','Length','Sequence','Supplimentary',
#                                'Comments','Type','Origin','Core',
#                                'ProgrammerRule','Submission','SRDMOrigin','Alias'])
#        
#    
#    with pd.ExcelWriter(path + 'spec.xlsx', mode='a') as writer:
#        df.to_excel(writer, sheet_name='Domainname' + val.upper())

In [ ]:
sheet = ps.Spreadsheet.create_new_sheet(5, 5)
# Set values
sheet.iloc[0, 0] = 25  # Set A1
sheet.iloc[1, 0] = sheet.iloc[0, 0]  # reference to A1
# Export to Excel
sheet.to_excel(path+'sample.xlsx')

In [ ]:
# importing tkinter and tkinter.ttk 
# and all their functions and classes 
from tkinter import * 
from tkinter.ttk import *

# importing askopenfile function 
# from class filedialog 
from tkinter.filedialog import askopenfile 

root = Tk() 
root.geometry('200x100') 

# This function will be used to open 
# file in read mode and only Python files 
# will be opened 
def open_file(): 
    file = askopenfile(mode ='r', filetypes =[('Excel Files', '*.xlsx')]) 
    if file is not None: 
        content = file.read() 
        print(content) 

btn = Button(root, text ='Open', command = lambda:open_file()) 
btn.pack(side = TOP, pady = 10) 

mainloop() 

In [ ]:
def path_manual():                          # This function is to create a gloabal variable namend folder
    global folder                           # so that we can use the path name outside function
    folder = filedialog.askdirectory()
    entry2.insert(0,folder)
    print("Path added")
    print(folder)
    

def path_file():                               #This is same as above but this functions ask for file name
    global file_xlsx
    file_xlsx = filedialog.askopenfilename(title = 'Select Specification Template File')
    print(file_xlsx)
    entry3.insert(0,file_xlsx)

In [ ]:
if __name__ == "__main__":
    #gui
    root = tk.Tk()  
    root.configure(background = 'white')
    root.title("SRDM to SDTM Mapping Specification")
    #root.geometry("700x450")
    
    canvas = tk.Canvas(root, height=HEIGHT, width=WIDTH)
canvas.pack()
    
    #Frame 2
    labelframe2 = LabelFrame(root, text = "Add Annotation",padx = 10,pady = 5,bg = '#95B9C7',font=('Helvetica',9,'bold'))  
    labelframe2.pack(fill="both", expand = "yes")  
    
    #annotation
    label2 = Label(labelframe2,text="Select Path", bg = '#95B9C7')
    label2.grid(row = 1, column = 1,padx = 10,pady = 12,sticky = W)
    button_inter = Button(labelframe2, text = "Choose file", bg = "#2B547E", 
                        fg = "white",command =  path_manual) # 
    button_inter.grid(row = 1,column = 2,padx = 10,pady =12,sticky = W)
    entry2 = Entry(labelframe2, width = 35, borderwidth=5)
    entry2.grid(row =1, column = 3,sticky = W )

    label3 = Label(labelframe2,text="Select Fille: ",bg = '#95B9C7')
    label3.grid(row = 2, column = 1,padx = 10,pady =12,sticky = W)
    button_pdf = Button(labelframe2, text = "Choose File", bg = "#2B547E", 
                        fg = "white", command = path_file )  #
    button_pdf.grid(row = 2, column = 2,sticky = W,padx = 10,pady =12)
    entry3 = Entry(labelframe2,width = 35, borderwidth=5)
    entry3.grid(row =2, column = 3,sticky = W )

    label3 = Label(labelframe2,text="Type Domain Names",bg = '#95B9C7')
    label3.grid(row = 3, column = 1,padx = 10,pady =12,sticky = W)
    q = StringVar()
    entry4 = Entry(labelframe2, textvariable = q , width = 35, borderwidth=5 )
    entry4.grid(row =3, column = 2,sticky = W )
    button_pdf = Button(labelframe2, text = "Extract Text", bg = "#2B547E", 
                    fg = "white",command = extract_text )  #
    button_pdf.grid(row = 3, column = 3,sticky = W,padx = 10,pady =12)


    # button_run2 = Button(labelframe2, text = "Run", bg = "#2B547E",fg= 'white') #
    # button_run2.grid(row = 7, column = 3, padx = 10,pady =5)
    # button_clear = Button(labelframe2, text = "Clear_all", bg = "#2B547E", fg= 'white', command = clear_all)  #
    # button_clear.grid(row =7, column =1)

    
    root.mainloop() 

In [ ]:
HEIGHT = 500
WIDTH = 600

def domain_list(fval):
    fval_list = np.unique([f.strip().upper() for f in fval.split(',') if fval]).tolist()  
    label['text'] = fval_list

root = tk.Tk()  
#root.configure(background = 'white')
root.title("SRDM to SDTM Mapping Specification")

canvas = tk.Canvas(root, height=HEIGHT, width=WIDTH)
canvas.pack()

#background_image = tk.PhotoImage(file=path+'pfizer2.png')
#background_label = tk.Label(root, image=background_image)
#background_label.place(relwidth=1, relheight=1)

frame = tk.Frame(root, bg='#80c1ff', bd=5)
frame.place(relx=0.5, rely=0.1, relwidth=0.75, relheight=0.1, anchor='n')

entry = tk.Entry(frame, font=40)
entry.place(relwidth=0.65, relheight=1)

button = tk.Button(frame, text="Execute", font=20, command=lambda: domain_list(entry.get()))
button.place(relx=0.7, relheight=1, relwidth=0.3)

lower_frame = tk.Frame(root, bg='#80c1ff', bd=10)
lower_frame.place(relx=0.5, rely=0.25, relwidth=0.75, relheight=0.6, anchor='n')

label = tk.Label(lower_frame)
label.place(relwidth=1, relheight=1)
root.mainloop() 

In [ ]:
def domain_list(fval):
    try:
        fval_list = np.unique(f.strip().upper() for f in fval.split(','))
    except:
        fval_list = 'Please Enter Comma Separated Domain Names to be Mapped to SDTM Specification'
    return fval_list
    print(fval_list)
      
    #label['text'] = fval_list

In [ ]:
HEIGHT = 500
WIDTH = 600

def test_function(entry):
    print("This is the entry:", entry)

def get_weather(entry):
    label['text'] = test_function(entry)



root = tk.Tk()

canvas = tk.Canvas(root, height=HEIGHT, width=WIDTH)
canvas.pack()

#background_image = tk.PhotoImage(file='landscape.png')
#background_label = tk.Label(root, image=background_image)
#background_label.place(relwidth=1, relheight=1)

frame = tk.Frame(root, bg='#80c1ff', bd=5)
frame.place(relx=0.5, rely=0.1, relwidth=0.75, relheight=0.1, anchor='n')

entry = tk.Entry(frame, font=40)
entry.place(relwidth=0.65, relheight=1)

button = tk.Button(frame, text="Get Weather", font=40, command=lambda: get_weather(entry.get()))
button.place(relx=0.7, relheight=1, relwidth=0.3)

lower_frame = tk.Frame(root, bg='#80c1ff', bd=10)
lower_frame.place(relx=0.5, rely=0.25, relwidth=0.75, relheight=0.6, anchor='n')

label = tk.Label(lower_frame)
label.place(relwidth=1, relheight=1)

root.mainloop()